## Import bibliotek.

In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols # regresja liniowa / metoda najmniejszych kwadratów STATSMODELS
from sklearn.linear_model import LinearRegression # regresja liniowa SKLEARN
from sklearn.model_selection import train_test_split # funkcja do podziału zbioru
from sklearn.metrics import mean_squared_error # metryka z której skorzystamy
import seaborn as sns # wizualizacje w Python
import matplotlib.pyplot as plt # wizualizacje w Python

## 1. Wczytanie zbiorów.

Dla przypomnienia - co udało nam sie ustalić podczas EDA:
* `ed` - najmocniejsza zmienna (widzieliśmy to w tabeli korelacji)
* `sex` - niezła zmienna (widzieliśmy to na wykresie gęstości dla zmiennej `earn`, którą podzieliliśmy względem płci)
* `sex`:`ed` - interakcja, którą odkryliśmy na tym wykresie (`sns.lmplot(data=wages, x='ed', y='earn', hue='sex', aspect=1.5)`)

In [2]:
wages = pd.read_csv('dane/wages.txt')
wages['height'] = wages['height'] * 2.54 # zamieńmy cale na cm
wages = wages.query('earn>1100') # usunięcie wartości odstających

In [3]:
wages['earn_log10'] = np.log10(wages.earn)
wages['age_log10'] = np.log10(wages.age)
wages['edu_level'] = wages.ed/wages.age # jaki procent swojego życia dana osoba spędziła na edukacji?

## 2. Podział zbioru.

UWAGA: podział wykonujemy ze względów dydaktycznych - by porównać wyniki dla zbioru uczącego i testowego + pokazać, że model przeucza/nieprzeucza. W rzeczywistej analizie sugerowałbym wykonanie podziału zbioru tuż przed szczegółową eksporacyjną analizą danych (EDA). Wtedy odkrywalibyśmy pewne relacje, badali wstępne hipotezy na zbiorze uczącym, a następnie weryfikowali je na zbiorze walidacyjnym.

Sugerowane podejście:
1. Odpowiednie wczytanie zbioru
    * zadbanie o poprawne wczytanie danych,
    * konwersja zmiennych do odpowiednich typów,
    * zbadanie braków danych, ich skali i wybranie sposobu ich osbłużenia.
2. Podział zbioru:
    * podstawowe statystyki, które mogą nam zasugerować sposób podziału (np. względem daty min. i maks.),
    * wybór strategii walidacyjnej (jak dzielimy zbiór, jakie proporcje, czy korzystamy ze stratyfikacji, etc.),
    * podział zbioru.
3. Eksploracyjna analiza danych.
    * badanie hipotez,
    * odkrywanie zależności,
    * odkrywanie interakcji,
    * budowa zmiennych.
4. Modelowanie.
    * uwzględnienie naszych odkryć w modelu,
    * próby ulepszenia modelu,
    * rzetelna walidacja,
    * budowa finalnego modelu.
5. Finalny test na zbiorze testowym.

In [4]:
wages_tr, wages_te = train_test_split(wages, train_size=0.75, random_state=2001)

In [5]:
wages.head()

,earn,height,sex,race,ed,age,earn_log10,age_log10,edu_level
0,79571.299011,187.6806,male,white,16,49,4.900756,1.690196,0.326531
1,96396.988643,168.2242,female,white,16,62,4.984063,1.792392,0.258065
2,48710.666947,161.9758,female,white,16,33,4.687624,1.518514,0.484848
3,80478.096153,160.5788,female,other,16,95,4.905678,1.977724,0.168421
4,82089.345498,160.2232,female,white,17,43,4.914287,1.633468,0.395349


## 3. Modelowanie.

### 3.0. Model 0.

In [6]:
wages_tr.earn.mean()

37860.99580200818

In [7]:
model_0 = ols(formula='earn ~ 1',
              data=wages_tr).fit() # dopasowujemy model do danych uczących / uczymy model

In [8]:
model_0.params # wyraz wolny z identycznym parametrem, jak średnia dla zmiennej celu!

Intercept    37860.995802
dtype: float64

In [9]:
model_0.summary() # podsumowanie dopasowanego modelu

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   earn   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 03 Aug 2023   Prob (F-statistic):                nan
Time:                        15:32:58   Log-Likelihood:                -10504.
No. Observations:                 894   AIC:                         2.101e+04
Df Residuals:                     893   BIC:                         2.101e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.786e+04   1025.390     36.924      0.000    3.58e+04    3.99e+04
==============================================================================
Omnibus:                      549.177   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6625.118
Skew:                           2.616   Prob(JB):                         0.00
Kurtosis:                      15.267   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Sprawdźmy jakośc modelu.

In [10]:
pred_tr = model_0.predict(wages_tr)
pred_te = model_0.predict(wages_te)

mse_tr = mean_squared_error(wages_tr.earn, pred_tr, squared=False)
mse_te = mean_squared_error(wages_te.earn, pred_te, squared=False)

print('MSE TR: {}'.format(np.round(mse_tr, 2)))
print('MSE TE: {}'.format(np.round(mse_te, 2)))

MSE TR: 30641.82
MSE TE: 31309.21


### 3.1. Model 1.
Zacznijmy od naszej najbardziej perspektywicznej zmiennej - `ed`. Zaczniemy modelowanie od zmiennej celu `earn`, by zademonstrować interpretację działania modelu regresji liniowej.

Pierwsze modele zbudujemy z użyciem klasy `ols` z biblioteki Statsmodels. Porównamy ją później z jej odpowiednikiem z `sklearn`.

In [11]:
model_1 = ols(formula='earn ~ ed',
              data=wages_tr).fit() # dopasowujemy model do danych uczących / uczymy model

In [12]:
model_1.summary() # podsumowanie dopasowanego modelu

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   earn   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     99.44
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           2.82e-22
Time:                        15:33:42   Log-Likelihood:                -10456.
No. Observations:                 894   AIC:                         2.092e+04
Df Residuals:                     892   BIC:                         2.093e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.843e+04   5728.109     -3.217      0.001   -2.97e+04   -7186.640
ed          4156.1827    416.789      9.972      0.000    3338.181    4974.185
==============================================================================
Omnibus:                      529.407   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6121.972
Skew:                           2.503   Prob(JB):                         0.00
Kurtosis:                      14.802   Cond. No.                         81.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Sprawdźmy jakośc modelu.

In [13]:
pred_tr = model_1.predict(wages_tr)
pred_te = model_1.predict(wages_te)

mse_tr = mean_squared_error(wages_tr.earn, pred_tr, squared=False)
mse_te = mean_squared_error(wages_te.earn, pred_te, squared=False)

print('MSE TR: {}'.format(np.round(mse_tr, 2)))
print('MSE TE: {}'.format(np.round(mse_te, 2)))

MSE TR: 29064.58
MSE TE: 28664.43


#### Co widzimy powyżej? Na co warto zwrócić uwagę?
* **`Prob (F-statistic)`** - jest to **p-value** dla statystyki F.
    * Pomaga nam ocnić, czy model, jako całość jest **istotny statystycznie**.
    * Mała wartość statystyki (< 0.05) oznacza, że model jest istotny statystycznie. Innymi słowy: istnieje statystycznie istotna zależność między zmiennymi objaśniającymi, a zmienną celu w modelu.
    * Im wartość statystyki bliższa 1, tym większe prawdopodobieństwo, że nie ma istotnej zależności pomiędzy zmiennymi objaśniającymi, a zmienną celu.
    * Naszym celem jest budowa modeli, które będą użyteczne. Dążymy do osiągnięcia niskiego poziomu "Prob (F-statistic)", mniejszego od 0.05. Tylko wtedy możemy być pewni wyników modelu i wnioskować na jego podstawie o relacjach pomiędzy zmiennymi objaśniającymi, a zmienną celu. 
* **`coef`** - współczynniki modelu dla poszczególnych zmiennych. Są one kolejnymi beta (beta_0, beta_1, itd.) z równania regresji liniowej.
* **`P>|t|`** - **p-value** (nazywana również "wartością p") dla poszczególnych współczynników modelu. Podobnie, jak w przypadku "Prob (F-statistic)":
    *  Mała wartość p-value (< 0.05) oznacza, że zaobserwowany efekt (wpływ zmiennej) jest istotny statystycznie. Innymi słowy: istnieje statystycznie istotna zależność między zmienną objaśniającą, a zmienną celu.
    * Im wartość p-value bliższa 1, tym większe prawdopodobieństwo, że nie ma istotnej zależności pomiędzy daną zmienną objaśniającą, a zmienną celu.
* **`[0.025 0.975]`** - 95-cio procentowe przedziały ufności dla poszczególnych współczynników modelu.
* **`Notes`** - automatycznie wygenerowane notatki dotyczące modelu. Warto zwrócić uwagę, czy nie ma w nich wskazanych żadnych istotnych uwag, lub ostrzeżeń dotyczących modelu.

Warto zwrócić uwagę, że model automatycznie dodał wyraz wolny ("Intercept"), pomimo iż nie ma go w "formule" modelu. Nie wszystkie implementacje regresji liniowej robią to w sposób automatyczny.

#### Czym jest istotność statystyczna?
Istotność statystyczna w modelu regresji liniowej odnosi się do tego, czy zmienna lub zbiór zmiennych objaśniających ma istotny wpływ na zmienną celu. Innymi słowy, chodzi o ocenę, czy zaobserwowane zależności wynikają z pewnych informacji zawartych w zbiorze, czy też wynikają z przypadku.

Istotność statystyczna występuje często w parze z prawdopodobieństwem, tzw. p-value. "Istotny" w tym kontekście oznacza, że istnieje niewielkie prawdopodobieństwo popełnienia błędu.

Przykład: powyższy model oszacował wartość współczynnika stojącego przy zmiennej `ed` na 4156.18. `P>|t|` (p-value) dla tego współczynnika wynosi 0.00. Oznacza to, że:
* istnieje statystycznie istotna zależność między zmienną objaśniającą, a zmienną celu (ponieważ p-value < 0.05),
* prawdopodobieństwo, że się mylimy (tj. zależność jednak nie istnieje, podczas, gdy my oceniamy, że istnieje) wynosi 0 (tyle ile wskazane p-value, które w istocie jest TYM prawdopodobieństwem).

#### Skąd akurat wartość jako wartość 0.05 rozgraniczająca pomiędzy istotnością statystyczną, a jej brakiem?
Wartość 0.05, to tzw. **`alfa`**, a więc zakładany poziom istotności, który jest powszechnie przyjmowany jako próg do podejmowania decyzji o odrzuceniu lub nieodrzuceniu hipotez statystycznych.

Alfa jest przyjmowany arbitralnie przez osobę przeprowadzającą analizę/budującą model, a jego wartośc równa 0.05 to przyjęty standard w statystyce.

#### Czym jest p-value? 
Jest to prawdopodobieństwo popełnienia błędu. Jeśli p-value < alfa (zakładany poziom istotności), to  możemy mówić o niewielkim prawdopodobieństwie popełnienia błędu, gdy stwierdzimy, że zaobserwowany efekt istnieje.

#### Czym jest istotność statystyczna?
Istotność statystyczna w modelu regresji liniowej odnosi się do tego, czy zmienna lub zbiór zmiennych objaśniających ma istotny wpływ na zmienną celu. Innymi słowy, chodzi o ocenę, czy zaobserwowane zależności wynikają z pewnych informacji zawartych w zbiorze, czy też wynikają z przypadku.

Istotność statystyczna występuje często w parze z prawdopodobieństwem, tzw. p-value. "Istotny" w tym kontekście oznacza, że istnieje niewielkie prawdopodobieństwo popełnienia błędu.

Przykład: powyższy model oszacował wartość współczynnika stojącego przy zmiennej `ed` na 4156.18. `P>|t|` (p-value) dla tego współczynnika wynosi 0.00. Oznacza to, że:
* istnieje statystycznie istotna zależność między zmienną objaśniającą, a zmienną celu (ponieważ p-value<alfa),
* prawdopodobieństwo, że się mylimy (tj. zależność jednak nie istnieje, podczas, gdy my oceniamy, że istnieje) wynosi 0.

#### Skąd akurat wartość jako wartość 0.05 rozgraniczająca pomiędzy istotnością statystyczną, a jej brakiem?
Wartość 0.05, to tzw. **`alfa`**, a więc zakładany poziom istotności, który jest powszechnie przyjmowany jako próg do podejmowania decyzji o odrzuceniu lub nieodrzuceniu hipotez statystycznych.

Alfa jest przyjmowany arbitralnie przez osobę przeprowadzającą analizę/budującą model, a jego wartośc równa 0.05 to przyjęty standard w statystyce.

#### Czym jest p-value? 
Jest to prawdopodobieństwo popełnienia błędu. Jeśli p-value < alfa (zakładany poziom istotności), to  możemy mówić o niewielkim prawdopodobieństwie popełnienia błędu, zakładając, że zaobserwowany efekt istnieje.

### Testowanie hipotez

Wszystko powyższe możemy odnieść to procesu testowania hipotez. Zawsze zakładamy w nim jakąś hipotezę zerową i hipoteze alternatywną. Nazwijny je kolejno: $H_0$ i $H_1$. Na starcie procesu testowania hipotez przyjmujemy $H_0$. Powszechnie przyjmuje się, że H_0 odnosi się do braku zależności/wpływu/różnicy.
![](zdjęcia/court.jpg)
Przykładem, który pomoże to zrozumieć jest sądownictwo. Na starcie procesu, zgodnie z zasadą domniemania niewinności uważa się oskarżonego za niewinnego. Dopiero w trakcie trwana procesu wysnuwane są oskarżenia i zbierane dowody na jego winę.

Podobnie jest w przypadku testowania hipotez. Na starcie zakłdamy brak związku ($H_0$). Nastpnie poszukujemy dowodów, które pozwalają nam odrzucić $H_0$ i przyjąć $H_1$. By to zrobić dowody muszą być przekonywujące i jednoznaczne. Zgodnie z przyjętym podejściem, maksymalny akceptowalne prawdopodobieństwo popełnienia błędu to 5% (zakładany poziom istotności alfa).

Podejście można streścić w następujących krokach:
1. Stawiamy hipotezy: $H_0$ i $H_1$.
2. Poszukujemy dowodów na prawdziwość $H_1$, odrzucenie $H_0$.
3. Wnioskujemy na podstawie zebranych dowodów (im więcej dowodów, tym p-value mniejsze):
    * jeśli p-value < alfa, to odrzucamy $H_0$ i przyjmujemy $H_1$,
    * jeśli p-value >= alfa, to nie mamy podstaw, by odrzucić $H_0$.
    
UWAGA: Brak dowodów na przyjęcie $H_1$ (p-value >= alfa) nie oznacza automatycznie, że $H_0$ jest prawdziwe. W takim przypadku nie akceptujemy ani $H_0$, ani $H_1$. $H_0$ pozostaje nieodrzucone.

### Wnioskowanie statystyczne (wnioskowanie parametryczne, na podstawie modelu).
W rozważanym przypadku dotyczącym zarobków w NY, nie jestesmy w stanie przeprowadzić ankiet z wszystkimi osobami zamieszkującymi NY. Badamy więc pewną **próbę** z populacji. We wnioskowaniu statystycznym chodzi o to, by powiedzieć coś na temat całej populacji, na podstawie próby, którą dysponujemy.

Znamy wzór regresji liniowej: $y = \beta_0 + \beta_1 x + \varepsilon$. Zdefiniujmy i odnieśmy do niego hipotezy w regresji liniowej:

* $H_0$ w przypadku parametrów regresji liniowej zakłada, że: $\beta_1 = 0$. Jeśli $\beta_1 = 0$ (prosta równoległa do osi x), to oznacza brak związku pomiędzy x i y ($0 x = 0$).
* $H_1$ w przypadku parametrów regresji liniowej zakłada, że: $\beta_1 \neq 0$.

By odrzucić $H_0$ należy wykluczyć możliwość, że prawdziwe $\beta_1=0$. Jak to zrobić? Model wyznacza przedziały ufności na poziomie 95% dla badango współczynnika, a następnie szacuje, jakie jest prawdopodobieństwo, że $\beta_1=0$ (dla całej populacji), jeśli zakładane przedziały są prawdziwe.

Przeanalizujmy przedziały ufności i współczynnik $\beta_1$, leżący przy zmiennej `ed`.
* współczynnik $\beta_1$ wynosi 4156.18,
* przedziały ufności napoziomie 95% wynoszą [3338.181, 4974.185].

Jest bardzo mało prawdopodobne, by prawdziwe $\beta_1$ (prawdziwe, tj. dla całej populacji) wynosiło 0.000, podczas gdy 95% przedział ufności wynosi [3338.181, 4974.185]. Przedział jest relatywnie wąski i leży "daleko" od zera, dlatego model wyznaczył p-value=0.000.

W związku z powyższymi kalkulacjami odrzucamy $H_0$ i przyjmujemy $H_1$ z maksymalnym dopuszczalym prawdopodobieństwem popełnienia błędu równy 5% (zakładany poziom istotności alfa, który nie postał przekroczony, bo p-value < alfa).

Gdybyśmy mieli ludzkim językiem wydać modelowi polecenie wyznaczenia p-value dla powyższego przykładu, to można by do niego powiedzieć:
> Wyznacz prawdopodobieństwo, że $\beta_1=4156.19$, przy założeniu, że prawdziwe (dla całej populacji) $\beta_1=0$.

W powyższym poleceniu każemy modelowi oszacować, jakie jest prawdopodobieństwo naszej pomyłki w ocenie parametru i relacji pomiędzy zmienną `ed`, a `earn`.
* Jeśli prawdopodobieństwo jest duże (>= 0.05), to nie można wykluczyć, że prawdziwe $\beta_1=0$. Prawdopodobieństwo jest nieakceptowalne duże, by mówić o statystycznie istotnej relacji pomiędzy `ed` i `earn`.
* Jeśli prawdopodobieństwo jest małe (< 0.05), to stwierdzamy, iż istnieją przesłanki, że prawdziwe  $\beta_1\neq0$. Prawdopodobieństwo popełnienia błędu jest akceptowalnie małe, dlatego mamy podstawy, by wierzyć w statystycznie istotną zalezność między `ed` i `earn`.

Dla zmiennej `ed` model wyznaczył prawdopodobieństwo (p-value) równe 0.000, dlatego są minimalne szanse, że prawdziwe $\beta_1=0$, podczas gdy zaobserwowane $\beta_1=4156.19$.

#### Jaka jest zatem relacja między poziomem edukacji (wyrażonym w latach edukacji), a zarobkami?

In [14]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   earn   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     99.44
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           2.82e-22
Time:                        15:34:05   Log-Likelihood:                -10456.
No. Observations:                 894   AIC:                         2.092e+04
Df Residuals:                     892   BIC:                         2.093e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.843e+04   5728.109     -3.217      0.001   -2.97e+04   -7186.640
ed          4156.1827    416.789      9.972      0.000    3338.181    4974.185
==============================================================================
Omnibus:                      529.407   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6121.972
Skew:                           2.503   Prob(JB):                         0.00
Kurtosis:                      14.802   Cond. No.                         81.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* Relacja pomiędzy poziomem edukacji, a zarobkami jest różna od 0.
* Edukacja istotnie wpływa na zarobki danej osoby.
* Wpływ ten jest istotny statystycznie.
* Na podstawie oszacowań modelu stwierdzamy, że jeden dodatkowy rok nauki średnio dodaje do rocznych zarobków 4156.18$.
* Mamy 95% ufności, że prawdziwa wartość tego współczynnika (wpływu edukacji na zarobki) leży pomiędzy 3338.181, a 4974.185.

### 3.2. Model 2.
Dodajmy do modelu kolejną zmienną, która była perspektywiczna (wg naszej oceny podczas EDA) - `sex`, czyli płeć.

In [15]:
model_2 = ols(formula='earn ~ ed + sex',
              data=wages_tr).fit()

In [16]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   earn   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.188
Method:                 Least Squares   F-statistic:                     104.6
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           1.62e-41
Time:                        15:34:23   Log-Likelihood:                -10409.
No. Observations:                 894   AIC:                         2.082e+04
Df Residuals:                     891   BIC:                         2.084e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -2.386e+04   5465.196     -4.365      0.000   -3.46e+04   -1.31e+04
sex[T.male]  1.863e+04   1874.665      9.939      0.000     1.5e+04    2.23e+04
ed           3981.5404    396.058     10.053      0.000    3204.225    4758.855
==============================================================================
Omnibus:                      524.161   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6584.654
Skew:                           2.435   Prob(JB):                         0.00
Kurtosis:                      15.371   Cond. No.                         81.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

`sex[T.male]` jest widoczne, a gdzie jest `sex[T.female]`? Zawarte w "Intercept". Kobiety w tym modelu stanowią tzw. *baseline*, czyli punkt odniesienia. Zawarcie kobiet, jako osobnej zmiennej doprowadziłoby prawdopodobnie do współliniowości. Czemu?
* Zmienne kategoryczne są reprezentowane w modelu po zmianie kodowania na tzw. *dummy coding*, a więc zmienne binarne.
    * W przypadku zmiennej kategorycznej o 2 poziomach potrzebujemy jednej zmiennej binarnej, by zawrzeć całą wiedzę, jaką ona niesie.
    * W przypadku zmiennej kategorycznej o 3 poziomach potrzebujemy dwóch zmiennych binarnych, by zawrzeć całą wiedzę, jaką ona niesie.
    * Itd.
* Gdy n-poziomów reprezentujemy w postaci n-zmiennych, to każdy z nich możemy ze 100% pewnością zamodelować poprzez pozostałe. Stąd w modelu pojawia się w tego typu przypadkach współliniowość.

Sprawdźmy jakośc modelu.

In [17]:
pred_tr = model_2.predict(wages_tr)
pred_te = model_2.predict(wages_te)

mse_tr = mean_squared_error(wages_tr.earn, pred_tr, squared=False)
mse_te = mean_squared_error(wages_te.earn, pred_te, squared=False)

print('MSE TR: {}'.format(np.round(mse_tr, 2)))
print('MSE TE: {}'.format(np.round(mse_te, 2)))

# model 1 = earn ~ ed
# MSE TR: 29064.58
# MSE TE: 28664.43

MSE TR: 27576.22
MSE TE: 28457.36


In [18]:
model_2.params

Intercept     -23857.872721
sex[T.male]    18631.489499
ed              3981.540418
dtype: float64

W powyższym modelu widać kilka interesujących informacji:
* `Prob (F-statistic):` jest jeszcze mniejsze, niż było	(2.82e-22 vs 1.62e-41). Model jest jeszcze bardziej istnotny.
* Zmieniło się oszacowanie parametru $\beta$ stojącego przy zmiennej `ed` (4156.1827 vs 3981.5404). Część wpływu na zarobki zabrała mu zmienna `sex`. Odnotujmy, że wraz ze zmianą postaci modelu (np. dodaniem/usunięciem zmiennych) mogą się zmienić oszacowania parametrów.
    * Interpretacja współczynnika przy zmiennej `ed`: dodatkowy rok nauki średnio dodaje do rocznych zarobków 3981.54\$.
    * Interpretacja współczynnika przy zmiennej `sex`: mężczyźni zarabiają średnio o 18631.49\$ więcej niż kobiety.
    
#### Ćwiczenie 1.
Jak oszacować zarobki dla mężczyzny, który przeszedł 16-letnią edukację?

$y = \beta_0 + \beta_1*sex + \beta_2*ed$

$y = -23857.87 + 18631.49*1 + 3981.54*16$ | przyjmujemy sex=1 dla mężczyzny, zgodnie ze wskazaniami modelu.

$y = 58478.26$

Mężczyzna, który kształcił się 16 lat zarabia ok. 58 478.26$.

#### Ćwiczenie 2.
Jak oszacować zarobki dla kobiety, która przeszła 20-letnią edukację?

$y = \beta_0 + \beta_1*sex + \beta_2*ed$

$y = -23857.87 + 18631.49*0 + 3981.54*20$ | przyjmujemy sex=1 dla mężczyzny, zgodnie ze wskazaniami modelu.

$y = 55772.93$

Kobieta, który kształciła się 20 lat zarabia ok. 55 772.93$.

### 3.3. Model 3.
Czy pamiętamy, że pod koniec EDA odkryliśmy interakcję pomiędzy `ed` i `sex`? Spróbujmy ją dodać do naszego modelu. By to zrobić, zamiast addytywnego wpływu zmiennych niezależnych (`earn ~ ed + sex`) uwzględniamy w modelu interakcję: `earn ~ ed:sex`, lub `earn ~ ed + sex`. Czym różnią się oba podejścia?

Z [dokumentacji Statsmodels](https://www.statsmodels.org/dev/example_formulas.html):
> “:” adds a new column to the design matrix with the product of the other two columns. “*” will also include the individual columns that were multiplied together.

In [ ]:
# przypomnienie wykresu
sns.lmplot(data=wages, x='ed', y='earn', hue='sex', aspect=1.3, height=4)
plt.show()

In [ ]:
model_3 = ols(formula='earn ~ ed:sex',
              data=wages_tr).fit()

In [ ]:
model_3.summary()

Sprawdźmy jakośc modelu.

In [ ]:
pred_tr = model_3.predict(wages_tr)
pred_te = model_3.predict(wages_te)

mse_tr = mean_squared_error(wages_tr.earn, pred_tr, squared=False)
mse_te = mean_squared_error(wages_te.earn, pred_te, squared=False)

print('MSE TR: {}'.format(np.round(mse_tr, 2)))
print('MSE TE: {}'.format(np.round(mse_te, 2)))

# model 1 = earn ~ ed
# MSE TR: 29064.58
# MSE TE: 28664.43

# model 2 = earn ~ ed + sex
# MSE TR: 27576.22
# MSE TE: 28457.36

In [ ]:
model_3.params

Szanowanie zarobków dla kobiety z 12-letnim wykształceniem:

$y = \beta_0 + \beta_1*ed*sex[female] + \beta_2*ed*sex[male]$

$y = -15550.992279 + 3368.676578*12*1 + 4730.911236*12*0$

$y = 24873.12$

Szanowanie zarobków dla mężczyzny z 5-letnim wykształceniem:

$y = \beta_0 + \beta_1*ed*sex[female] + \beta_2*ed*sex[male]$

$y = -15550.992279 + 3368.676578*5*0 + 4730.911236*5*1$

$y = 8103.56$

In [ ]:
# Potwierdźmy powyższe wyniki z pomocą modelu.
model_3.predict(pd.DataFrame(
    {
        'ed': [12, 5],
        'sex': ['female', 'male']
    }))

### 3.4. Model 4.

Sprawdźmy, czy możemy dodać do modelu kolejną zmienną, np. `race`. Podczas EDA wielokrotnie przedziały ufności wskazywały na trudności w oszacowaniu jest realnego związku ze zmienną celu. Czy model zareaguje podobnie?

In [ ]:
wages['is_white'] = np.where(wages.race=='white',
                             1,
                             np.NaN)

In [ ]:
model_4 = ols(formula='earn ~ ed:sex + race',
              data=wages_tr).fit()

In [ ]:
model_4.summary()

Okazuje się, że współczynniki dla tej zmiennej są nieitotne statystycznie. Powinniśmy ją usunąć z modelu.

Pytanie: gdzie jest poziom "black" dla zmiennej `race`?

### 3.5. Model 5.

Widzimy, że najbliżej istotności był poziom "white". Czy możemy taki efekt "wyłuskać" i zawrzeć w modelu? Tak! :-)

In [ ]:
wages_tr = wages_tr.copy()
wages_te = wages_te.copy()

In [ ]:
wages_tr['race_white'] = wages_tr.race=='white'
wages_te['race_white'] = wages_te.race=='white'

In [ ]:
model_5 = ols(formula='earn ~ ed:sex:race_white',
              data=wages_tr).fit()
# Najlepsze połączenie to interakcja trzech powyższych czynników.

In [ ]:
model_5.summary()

Sprawdźmy jakośc modelu.

In [ ]:
pred_tr = model_5.predict(wages_tr)
pred_te = model_5.predict(wages_te)

mse_tr = mean_squared_error(wages_tr.earn, pred_tr, squared=False)
mse_te = mean_squared_error(wages_te.earn, pred_te, squared=False)

print('MSE TR: {}'.format(np.round(mse_tr, 2)))
print('MSE TE: {}'.format(np.round(mse_te, 2)))

# model 1 = earn ~ ed
# MSE TR: 29064.58
# MSE TE: 28664.43

# model 2 = earn ~ ed + sex
# MSE TR: 27576.22
# MSE TE: 28457.36

# model 3 = earn ~ ed:sex
# MSE TR: 27554.5
# MSE TE: 28245.83

### 3.6. Model 6.

Na początku notatnika dodaliśmy zmienną `edu_level`. Czy jest ona istotna? Sprawdźmy...

In [ ]:
model_6 = ols(formula='earn ~ ed:sex:race_white + edu_level',
              data=wages_tr).fit()

In [ ]:
model_6.summary()

Sprawdźmy jakośc modelu.

In [ ]:
pred_tr = model_6.predict(wages_tr)
pred_te = model_6.predict(wages_te)

mse_tr = mean_squared_error(wages_tr.earn, pred_tr, squared=False)
mse_te = mean_squared_error(wages_te.earn, pred_te, squared=False)

print('MSE TR: {}'.format(np.round(mse_tr, 2)))
print('MSE TE: {}'.format(np.round(mse_te, 2)))

# model 1 = earn ~ ed
# MSE TR: 29064.58
# MSE TE: 28664.43

# model 2 = earn ~ ed + sex
# MSE TR: 27576.22
# MSE TE: 28457.36

# model 3 = earn ~ ed:sex
# MSE TR: 27554.5
# MSE TE: 28245.83

# model 5 = earn ~ ed:sex:race_white
# MSE TR: 27197.69
# MSE TE: 28052.97

# model 6 = earn ~ ed:sex:race_white + edu_level
# MSE TR: 26361.12
# MSE TE: 27237.29

#### Porównanie wyników kolejnych modeli.

In [ ]:
df = pd.DataFrame({'MSE': [31309.21, 28664.43, 28457.36, 28245.83, 28052.97, 27237.29]})

In [ ]:
df['MSE_IMP'] = df.diff()

In [ ]:
df['MSE_IMP_PRC'] = -100*df.MSE_IMP/df.MSE

In [ ]:
df

## 4. Dodatkowe hipotezy.

### 4.1. Wzrost, a zarobki.
Jak to jest z tym wzrostem? Czy wpływa na zarobki, czy też nie? Sprawdźmy to.

$H_0$ - wzrost nie wpływa na zarobki.
$H_1$ - wzrost wpływa na zarobki.

In [ ]:
model_wzrost_1 = ols(formula='earn ~ height',
                   data=wages_tr).fit()
model_wzrost_1.summary()

* p-value < 0.05 - istnieje statystycznie istotna relacja między wzrostem, a zarobkami, prawda?
* Każdy dodatkowy cm wzrostu, to ok 848 dolarów do rocznej pensji.
* Zwróć jednak uwagę na sekcję "Notes". Mamy w niej ostrzeżnie o "współliniowości lub innym problemie numerycznym". Współliniowość z jedną zmienną?

In [ ]:
model_wzrost_2 = ols(formula='earn ~ np.log(height)',
                   data=wages_tr).fit()
model_wzrost_2.summary()

Dodajmy ją zatem do naszego modelu, skoro jest istotna.

In [ ]:
model_7 = ols(formula='earn ~ ed:sex:race_white + edu_level + np.log(height)',
              data=wages_tr).fit()
model_7.summary()

Jednak jest nieistotna i dodatkowo współczynnik różni się znacząco (1.44e+05 vs 3.219e+04). :( Po co więc cała statystyka i mówienie o tym, że mamy prawo coś wnioskować, skoro wystarczy dodać nowe zmienne i wszystko się zmienia? Jak to wytłumaczyć?

##### Paradoks Simpsona - opis.
Powyższe zjawisko nazywa się **paradoksem Simpsona**. Może ono występować m.in. w modelowaniu. Dochodzi w nim do zmiany relacji między zmiennymi, w wyniku wprowadzenia dodatkowej zmiennej do modelu regresji.

Efekt ten może prowadzić do odmiennych wniosków dotyczących związku między zmiennymi w zależności od tego, czy analiza jest przeprowadzana dla całego zestawu danych lub podzielonego na grupy.

##### Paradoks Simpsona - przyczyna.
Przyczyną tego zjawiska są różnice w rozkładach zmiennych między grupami, które to mogą wpływać na wyniki analizy.

##### Paradoks Simpsona - rozwiązanie.
* Badanie różnych zestawów zmiennych i notowanie, która zmienna najbardziej wpływa na model.
* Wnikliwa interpretacja wyników modelu, szczególnie w przypadku, gdy dane są podzielone na różne grupy.

### 4.2. Regresja liniowa w sklearn.

Spróbujmy odtworzyć np. ten model: `earn ~ ed + sex`.

In [ ]:
wages_tr

In [ ]:
model_sklearn = LinearRegression()
# model_sklearn.fit(wages_tr[['ed', 'sex']], wages_tr[['earn']]) # ta linia zwróci błąd

Wszystkie dane, jakich używamy w modelach z biblioteki sklear, muszą być przygotowanie i sprowadzone do postaci wartości numerycznych.

In [ ]:
wages_tr['sex_male'] = pd.get_dummies(wages_tr.sex, drop_first=True)
wages_te['sex_male'] = pd.get_dummies(wages_te.sex, drop_first=True)

In [ ]:
model_sklearn.fit(wages_tr[['ed', 'sex_male']], wages_tr[['earn']])

In [ ]:
# Podejrzyjmy co możemy z niego wyciągnąć.
dir(model_sklearn)

Wyciągnijmy z niego współczynniki

In [ ]:
model_sklearn.intercept_

In [ ]:
model_sklearn.coef_

Poniżej, dla przypomnienia, współczynniki modelu 2.
```
Intercept     -23857.872721
sex[T.male]    18631.489499
ed              3981.540418
```
Są identyczne, jednak sam sposób budowania modelu dosyć znacząco się różni.
* Nie możemy definiować modelu poprzez "formułę".
* Musimy obsłużyć zmienne kategoryczne.
* Musimy obsłużyć braki danych.
* Nie mamy dostępu do p-values dla modelu. Nie wiemy zatem, czy model jest istotny, czy nie.
* Nie mamy dostępu do p-values dla poszczególnych zmiennych. Nie wiemy zatem, które zmienne są istotne, a które nie.

Część powyższych da się obejść, ale jest to nieco problematyczne. Przykład dla modelu 6:

In [ ]:
from patsy import dmatrices
formula = 'earn ~ ed:sex:race_white + edu_level'
y, X = dmatrices(formula, wages_tr, return_type='dataframe')

model_sklearn.fit(X, y)

In [ ]:
model_sklearn.intercept_

In [ ]:
model_sklearn.coef_

In [ ]:
model_6.params # przypomnienie parametrów dla modelu 6

### 4.3. Selekcja zmiennych w regresji liniowej.
1. **Podejście eksperckie, bazowanie na wiedzy branżowej** - polega na skonsultowaniu się z ekspertem w danej dziedzinie (chyba że sami nim jesteśmy).
2. **Metody automatycznej selekcji zmiennych**  - istnieją wpubowane w regresję liniową metody selekcji zmiennych, np. LASSO, która automatycznie usuwa ze zbioru nadmiarowe zmienne.
    * Lasso w sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
    * Lasso w statsmodels: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.fit_regularized.html
3. **Bazująca na danych** - istnieją krokowe metody selekcji zmiennych, np. .
    * Wybierasz statystykę, którą mierzysz dopasowanie modelu do danych.
    * *Backward selection* - zaczynając od modelu ze wszystkimi zmiennymi, następnie redukujesz kolejne (najsłabsze) zmienne tak długo, jak długo poprawia się wynik modelu.
    * *Forward selection* - zaczynając od pustego modelu, następnie dodajesz kolejne (najmnocniejsze) zmienne tak długo, jak długo poprawia się wynik modelu.
    * Oba powyższe podejścia są dostępne:
        * w bibliotece `mlxtend`: https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.feature_selection/#sequentialfeatureselector
        * w bibliotece `sklearn`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html

![](zdjęcia/stepwise.png)

Źródła:
* Zdjęcie: https://pixabay.com/illustrations/justice-straight-jurisdiction-2071539/
* Paradoks Simpsona: https://pl.wikipedia.org/wiki/Paradoks_Simpsona
* Forward Stepwise Selection: https://github.com/arpanganguli/ISLP/blob/master/Chapter%207/Applied%20Exercises/10.ipynb